In [0]:
# ALUMNO: Víctor Manuel Cavero Gracia 

import numpy as np

# Generacion de las matrices

#m1 = np.array([[1,2,3], [4,5,6], [7,8,9]])
#m2 = np.array([[1,2,3], [4,5,6], [7,8,9]])

m1 = np.array([[1,2], [3,4]])
m2 = np.array([[1,2], [3,4]])

In [0]:
# Creamos el RDD 
rdd1 = spark.sparkContext.parallelize((m1,m2))

print(rdd1.collect())

[array([[1, 2],
 [3, 4]]), array([[1, 2],
 [3, 4]])]

In [0]:
# Uso de flat map para asignar el indice a cada fila
rdd2 = rdd1.flatMap(lambda x: [(y, x[y]) for y in range(0, len(x))])

print(rdd2.collect())

[(0, array([1, 2])), (1, array([3, 4])), (0, array([1, 2])), (1, array([3, 4]))]

In [0]:
import functools

def suma(a,b):
  t = []
  l = len(a)
  for x in range(0,l):
    t.append(a[x] + b[x])
  return t

def resta(a,b):
  t = []
  l = len(a)
  for x in range(0,l):
    t.append(a[x] - b[x])
  return t


# Para printear la matriz final resultante
def printMatrix(m):
  for (x,y) in m: print(y)

In [0]:
#Operacion final juntando los resultados mediante reduceKey y la operacion seleccionada

rdd3 = rdd2.reduceByKey(lambda a,b: suma(a,b))

for x in m1: print(x)
print( ' + ')
for x in m2: print(x)
print( ' = ')
printMatrix(rdd3.collect())

[1 2]
[3 4]
 + 
[1 2]
[3 4]
 = 
[2, 4]
[6, 8]

In [0]:
rdd4 = rdd2.reduceByKey(lambda a,b: resta(a,b))

for x in m1: print(x)
print( ' - ')
for x in m2: print(x)
print( ' = ')
printMatrix(rdd4.collect())

[1 2]
[3 4]
 - 
[1 2]
[3 4]
 = 
[0, 0]
[0, 0]

In [0]:
def modify_m1(m,f):
  for x in range(0, len(m)): 
    m[x] = m[f]
  return m

def mult(x,y):
  sum = 0
  for z in range(0, len(x)):
    sum = sum + (x[z]*y[z])
  return sum

def multiplicacion(m1,m2): 
  m3 = m2.transpose()
  rdds = []
  result = []
  final_result = []
  for x in range(0,len(m1)):
    m1_mod = modify_m1(m1.copy(), x)
    rddx1 = spark.sparkContext.parallelize((m1_mod,m3))
    rdds.append(rddx1.flatMap(lambda x: [(y, x[y]) for y in range(0, len(x))]))
  for x in rdds:
    result.append(x.reduceByKey(lambda a,b: mult(a,b)))
  for x in result: 
    sub_mult = []
    for (y,z) in x.collect():
      sub_mult.append(z)
    final_result.append(sub_mult)
  return final_result

In [0]:
for x in m1: print(x)
print( ' * ')
for x in m2: print(x)
print( ' = ')
for x in multiplicacion(m1,m2): print(x)

[1 2]
[3 4]
 * 
[1 2]
[3 4]
 = 
[7, 10]
[15, 22]

In [0]:
# Division = A / B = A * B ^ -1

m2_inverse = np.linalg.inv(m2)

for x in m1: print(x)
print( ' / ')
for x in m2: print(x)
print( ' = ')
for x in multiplicacion(m1,m2_inverse): print(x)

[1 2]
[3 4]
 / 
[1 2]
[3 4]
 = 
[1.0, 0.0]
[8.881784197001252e-16, 0.9999999999999996]